# The intent of this notebook is to familiarize ourselves with the data, do some exploratory analysis and to construct timeseries and other data for further analysis by the team

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import datetime
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon

#set default plot characterstics and colors
from matplotlib import rcParams

import prep_data

dark_colors = ["#99D699", "#B2B2B2",
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843),
                (0.4, 0.4, 0.4)]

rcParams['figure.figsize'] = (10, 8)
rcParams['figure.dpi'] = 150
rcParams['axes.color_cycle'] = dark_colors
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = "white"
rcParams['axes.titlesize'] = 20      
rcParams['axes.labelsize'] = 17.5
rcParams['xtick.labelsize'] = 15 
rcParams['ytick.labelsize'] = 15
rcParams['legend.fontsize'] = 17.5
rcParams['patch.edgecolor'] = 'none'
rcParams['grid.color']="white"   
rcParams['grid.linestyle']="-" 
rcParams['grid.linewidth'] = 1
rcParams['grid.alpha']=1
rcParams['text.color'] = "444444"
rcParams['axes.labelcolor'] = "444444"
rcParams['ytick.color'] = "444444"
rcParams['xtick.color'] = "444444"

# import findspark
# findspark.init()
# import pyspark
# sc = pyspark.SparkContext(appName="Spark1")

/Users/Sam/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


### Loading the data with some initial cleaning (h/t CS109Hubway for getting me started on this and for having the data loaded to github already)

In [2]:
stations_df = prep_data.stations('https://raw.githubusercontent.com/CS109Hubway/classp/master/data/hubway_stations.csv')
stations_df.head()

,id,terminal,station,municipal,lat,lng,status
0,3,B32006,Colleges of the Fenway,Boston,42.340021,-71.100812,Existing
1,4,C32000,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616,Existing
2,5,B32012,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179,Existing
3,6,D32000,Cambridge St. at Joy St.,Boston,42.361285,-71.065140,Existing
4,7,A32000,Fan Pier,Boston,42.353412,-71.044624,Existing


In [3]:
stations_df.shape

(142, 7)

In [4]:
station_ids = stations_df['id'].unique()

In [5]:
trips_df = prep_data.trips('https://raw.githubusercontent.com/CS109Hubway/classp/master/data/tripsthrough2012.csv')
trips_df.head()

,index,seq_id,hubway_id,status,duration,start_date,strt_statn,end_date,end_statn,bike_nr,...,st_hour,end_hour,st_minute,end_minute,st_month,end_month,st_daydate,end_daydate,st_weekday,end_weekday
0,140521,140522,157655,Closed,189,2012-03-13 18:31:00,29,2012-03-13 18:34:00,18,B00094,...,18,18,1111,1114,3,3,2012-03-13,2012-03-13,1,1
1,140522,140524,157657,Closed,704,2012-03-15 00:00:00,40,2012-03-15 00:12:00,40,B00082,...,0,0,0,12,3,3,2012-03-15,2012-03-15,3,3
2,140523,140523,157656,Closed,1700,2012-03-15 00:00:00,7,2012-03-15 00:28:00,15,B00603,...,0,0,0,28,3,3,2012-03-15,2012-03-15,3,3
3,140524,140525,157658,Closed,1411,2012-03-15 00:00:00,41,2012-03-15 00:24:00,36,B00278,...,0,0,0,24,3,3,2012-03-15,2012-03-15,3,3
4,140525,140526,157659,Closed,251,2012-03-15 00:03:00,43,2012-03-15 00:07:00,40,B00076,...,0,0,3,7,3,3,2012-03-15,2012-03-15,3,3


In [6]:
# Assign Time Periods:
# 1: Morning (4am-12pm)
# 2: Afternoon (12pm-8pm)
# 3: Evening (8pm-4am)
time_periods = np.zeros(len(trips_df))
start_dates = trips_df['start_date'].values
for i in xrange(len(start_dates)):
    hour = pd.Timestamp(start_dates[i]).time().hour
    if 4 <= hour and hour < 12:
        #morning
        time_period = 1
    elif 12 <= hour and hour < 20:
        # afternoon
        time_period = 2
    else:
        # evening 
        time_period = 3
    time_periods[i] = time_period

In [7]:
trips_df['time_period'] = time_periods
trips_df.head()

,index,seq_id,hubway_id,status,duration,start_date,strt_statn,end_date,end_statn,bike_nr,...,end_hour,st_minute,end_minute,st_month,end_month,st_daydate,end_daydate,st_weekday,end_weekday,time_period
0,140521,140522,157655,Closed,189,2012-03-13 18:31:00,29,2012-03-13 18:34:00,18,B00094,...,18,1111,1114,3,3,2012-03-13,2012-03-13,1,1,2
1,140522,140524,157657,Closed,704,2012-03-15 00:00:00,40,2012-03-15 00:12:00,40,B00082,...,0,0,12,3,3,2012-03-15,2012-03-15,3,3,3
2,140523,140523,157656,Closed,1700,2012-03-15 00:00:00,7,2012-03-15 00:28:00,15,B00603,...,0,0,28,3,3,2012-03-15,2012-03-15,3,3,3
3,140524,140525,157658,Closed,1411,2012-03-15 00:00:00,41,2012-03-15 00:24:00,36,B00278,...,0,0,24,3,3,2012-03-15,2012-03-15,3,3,3
4,140525,140526,157659,Closed,251,2012-03-15 00:03:00,43,2012-03-15 00:07:00,40,B00076,...,0,3,7,3,3,2012-03-15,2012-03-15,3,3,3


In [8]:
del time_periods
del start_dates

In [9]:
# Create masks for each time period
morning_mask = trips_df['time_period'] == 1
afternoon_mask = trips_df['time_period'] == 2
evening_mask = trips_df['time_period'] ==3

In [10]:
# Let K = maxmimum id of a station
K = stations_df.id.max()

In [16]:
# drop rows with NAs
trips_df.dropna(axis=0,inplace=True)

In [57]:
mini_trips = trips_df.loc[:,["strt_statn","end_statn","time_period"]]
transition_matrix = np.zeros([146,146,3])
for index, row in mini_trips.dropna(axis=0).iterrows():
    print "\r" + str(index) + " " + str(row[0]) + " " + str(row[1]),
    transition_matrix[row[0],row[1],row[2]-1] += 1
for period in range(3):
    for i in range(146):
        transition_matrix[i,:,period] = transition_matrix[i,:,period] / np.sum(transition_matrix[i,:,period])

408764 55.0 45.0


/Users/Sam/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [60]:
sum(transition_matrix[4,:,1])

1.0000000000000004

In [66]:
# Create distance and duration matrices

trips_df["trip_duration"] = trips_df["end_minute"] - trips_df['st_minute']
mini_trips = trips_df.loc[:,["strt_statn","end_statn","trip_duration"]]
mini_trips = mini_trips.dropna(axis=0)
duration_matrix = np.zeros([max(stations_df.id)+1,max(stations_df.id)+1])
counter_matrix = np.zeros([max(stations_df.id)+1,max(stations_df.id)+1])
distance_matrix = np.zeros([max(stations_df.id)+1,max(stations_df.id)+1])
# create duration matrix
for index, row in mini_trips.iterrows():
    print "\r" + str(index) + " " + str(row[0]) + " " + str(row[1]),
    if row[2] > 0:
        duration_matrix[row[0],row[1]] += row[2]
        counter_matrix[row[0],row[1]] += 1.0

duration_matrix /= counter_matrix
# create distance matrix
for index, row in stations_df.iterrows():
    for index2, row2 in stations_df.iterrows():
        print "\r" + str(row[0]) + " " + str(row2[0]),
        station1 = row[0]
        station2 = row2[0]
        phi1 = row[4]*np.pi/180.
        phi2 = row2[4]*np.pi/180.
        lam1 = row[5]*np.pi/180.
        lam2 = row2[5]*np.pi/180.
        delphi = np.abs(phi1 - phi2)
        dellam = np.abs(lam1 - lam2)
        delsig = np.arccos(np.sin(phi1)*np.sin(phi2)+np.cos(phi1)*np.cos(phi2)*np.cos(dellam))
        r = 3959 #mi
        distance_matrix[station1,station2] = r*delsig    

145 145


/Users/Sam/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Sam/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [67]:
# Get probability that there is a rider at a given station at a given time period
rides = np.zeros((3,max(stations_df.id)+1))
for index, row in trips_df.iterrows():
    print "\r" + str(index) + " " + str(row[6]) + " " + str(row[-2]),
    rides[row[-2]-1,row[6]] += 1

408764 55.0 3.0


/Users/Sam/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [68]:
# Simulated Annealing

In [70]:
# create a new initial bike positions
# returns the new starting positions
def change_mapping(bike_start_locs,num_to_move):
    # make copy
    proposed_bike_start_locs = np.copy(bike_start_locs)
    # Choose which bike to move
    bikes_to_move = np.random.choice(np.arange(len(bike_start_locs)),replace=False,size=num_to_move)
    for bike_idx in bikes_to_move:
        # choose where to move bike to
        move_bike_to = np.random.randint(3,K+1)
        proposed_bike_start_locs[bike_idx] = move_bike_to
    return proposed_bike_start_locs

In [91]:
def runday(proposed_bike_start_locs, interval, ride_probs):
    # Initialize matrix to store bike data (changing each interval)
    # 0th column is whether the bike is in transit (1= in transit)
    # 1st col is time left in transit
    # 2nd column is location
    # 3rd column is total distance for bike in day
    bikes = np.zeros((len(proposed_bike_start_locs),4))
    #intial time
    t = 0
    bikes[:,2] = proposed_bike_start_locs
    morning_begin = 240
    afternoon_begin = 720
    evening_begin = 1200
    while t < 1440:
        #determine current time period:
        current_hour = t % 60
        time_period = 2
        if current_hour >= 4 and current_hour < 12:
            time_period = 0
        elif current_hour >=12 and current_hour < 20:
            time_period = 1
        for i in xrange(bikes.shape[0]):
            bike = bikes[i]
            if bike[0] == 1:
                # in transit
                bike[1] -= interval
                if bike[1] <= 0:
                    # if time left is zero
                    #then bike is not in transit
                    bike[0] = 1
            else:
                # not in transit
                if np.random.random() < ride_probs[time_period,bike[2]]:
                    # a rider wants the bike
                    prev_loc = bike[2]
                    bike[2] = np.random.choice(np.arange(0,K+1),p=transition_matrix[prev_loc,:,time_period])
                    #update duration
                    bike[1] = duration_matrix[prev_loc,bike[2]]
                    #update distance
                    bike[3] = distance_matrix[prev_loc,bike[2]]
                    bike[0] = 1
        t += interval
    return np.sum(bikes[:,3])

In [106]:
# function for simulated annealing
# x: points1
# y: points2
# w: array of weights
# v: array of values
# total_weight_limit: total weight limit
# init_temp: initial temperature 
# thermostat: linear factor to decrease the temperature 
# ftol, itol: tolerance values for stopping 
# ftol: Not changing much anymore --> function tolerance threshold
# itol: maximum number of iterations
# reannealing: schedule for reheating

def simulated_annealing(init_temp, thermostat, ftol, itol, reannealing,num_bikes,interval):
    # number of iterations before stopping
    # if improvment is less than function tolerance, typically m = 100
    m=100
    
    # initialize correspondence matrix with random mappings
    bike_start_locs = np.random.randint(3,K+1,size=num_bikes)
    # array storing the norm at each step
    total_dists = []
    
    # initialize temperature
    temperature = init_temp   

    # number of accepted steps
    it = 0                    
    A = np.zeros((3,3))
    # Energy is the norm
    prev_dist = 0
    # number of iterations
    atp=0
    
    # keep track of most valuable knapsack
    best_bike_start_locs = np.copy(bike_start_locs)
    best_dist = prev_dist
    
    #normalize probability of a rider taking a bike for the given interval length
    ride_probs = rides/8.0/60.0*interval
    
    while it >= 0:
        # adaptive step size 
        num_to_move = np.max((np.floor(temperature).astype(int),2))
        #num_to_move = 2
        proposed_bike_start_locs = change_mapping(bike_start_locs,num_to_move)
        
        # make energy negative, so we minimize energy
        new_dist = runday(proposed_bike_start_locs,interval,ride_probs)
        delta_dist =  new_dist - prev_dist 

        if new_dist > prev_dist:
            bike_start_locs = proposed_bike_start_locs
            total_dists.append(new_dist)
            prev_dist = new_dist 
            it = it+1
            
        elif np.random.rand() < np.exp( delta_dist/temperature):
            bike_start_locs = proposed_bike_start_locs
            total_dists.append(new_dist)
            prev_dist = new_dist
            it = it+1
        
        # Store most valuable knapsack
        if new_dist > best_dist:
            best_bike_start_locs = np.copy(proposed_bike_start_locs)
            best_dist = new_dist
            
        atp = atp +1  # NUMBER OF ITERATIONS
        
        # check if it is time to cool down
        if it % reannealing == 0:
            temperature = thermostat * temperature
            compl_temp=0
            
            #if we get too cold, reheat
            if temperature < 0.01:
                temperature = 1
                
        if len(total_dists)>m and np.std(total_dists[-m:])/np.mean(total_dists[-m:]) < ftol:
            print 'ftol'
            break
        if atp >itol:
            print 'itol'
            break
        print '\r'+str(it),

    return best_bike_start_locs, best_dist

In [122]:
init_temp = 10.0
num_bikes = 50
thermostat = .9
ftol = .01
itol = 1000
reannealing =10
interval = 10
best_bike_start_locs,best_dist = simulated_annealing(init_temp, thermostat, ftol, itol, reannealing,num_bikes,interval)
print best_bike_start_locs
print best_dist

75 itol
[108  11 103  57  53  12   7 134  41  44  94  18  31  79  71  85 143  76
 131  35  68 126  57  77  15  72   6 136  89  69 108  66  28  32  86 119
  56  82 145  40  10  43 134  37  56  58  23 106 108  25]
58.7393583077


/Users/Sam/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Sam/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:36: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Sam/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Sam/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:40: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
